# Frozen OpenCLIP

> Interface to the [OpenCLIP](https://github.com/mlfoundations/open_clip) library.

In [ ]:
#| default_exp models.frozen_open_clip

In [ ]:
#| export
from genQC.imports import *
from genQC.models.config_model import ConfigModel
from genQC.utils.async_fn import run_parallel_jobs
from genQC.utils.misc_utils import infer_torch_device
import open_clip

In [ ]:
print("OpenCLIP version:", open_clip.__version__)

OpenCLIP version: 2.32.0


## CLIP model

In [ ]:
#| export
@dataclass
class FrozenOpenCLIPEmbedderConfig:
    arch: str
    version: str
    #device: str
    max_length: int
    freeze: bool
    layer: str

In [ ]:
#| export
class FrozenOpenCLIPEmbedder(ConfigModel):
    """Loads and freezes the [OpenCLIP](https://github.com/mlfoundations/open_clip) transformer encoder for text prompts."""
    
    LAYERS = [
        # "pooled",
        "last",
        "penultimate"
    ]

    njobs = 1

    def __init__(self, arch="ViT-B-32", version="datacomp_xl_s13b_b90k", max_length=77, freeze=True, layer="penultimate", **kwargs):
        super().__init__()        
        
        assert layer in self.LAYERS     
        self.params_config = FrozenOpenCLIPEmbedderConfig(arch, version, max_length, freeze, layer)
        
        model, _, _ = open_clip.create_model_and_transforms(arch, device="cpu", pretrained=version)
        self.device = "cpu"
        
        del model.visual     
        self.model = model
        # self.to(device)
        
        self.tokenizer = open_clip.get_tokenizer(arch)
        assert torch.numel(self.tokenizer("test"))
        
        assert max_length <= 77   # max set by the clip 
        self.max_length = max_length
        
        if freeze: self.freeze()
        
        self.layer = layer
        if   self.layer == "last":         self.layer_idx = 0
        elif self.layer == "penultimate":  self.layer_idx = 1
        else: raise NotImplementedError()

        #create empty token, can also be, e.g., A nice picture
        self.empty_token = self.tokenize_and_push_to_device("")
        
    def freeze(self, freeze: bool = True):
        super().freeze(freeze=freeze)
                    
        for param in self.model.parameters(): 
            param.requires_grad = not freeze
    
    def to(self, device):
        self.model  = self.model.to(device)           
        self.device = device
        return self
        
    @torch.inference_mode()
    def tokenize_and_push_to_device(self, text, to_device=True):
        if self.njobs > 1:

            tokens_list = run_parallel_jobs(self.tokenizer, np.array_split(text, self.njobs), self.njobs)
            tokens      = torch.cat(tokens_list, dim=0)
            
        else:
            # tokens = open_clip.tokenize(text)
            tokens = self.tokenizer(text)
        
        if to_device:
            tokens = tokens.to(self.device)
        return tokens
    
    @torch.inference_mode()
    def forward(self, c, **kwargs):
        return self.encode_with_transformer(c)

    @torch.inference_mode()
    def encode_with_transformer(self, text):
        cast_dtype = self.model.transformer.get_cast_dtype()
        
        x = self.model.token_embedding(text).to(cast_dtype)  # [batch_size, n_ctx, d_model]        
        x = x + self.model.positional_embedding[None, :x.shape[1]].to(cast_dtype)

        if not self.model.transformer.batch_first:
            x = x.permute(1, 0, 2)  # NLD -> LND
        
        x = self.text_transformer_forward(x, attn_mask=self.model.attn_mask)

        if not self.model.transformer.batch_first:
            x = x.permute(1, 0, 2)  # LND -> NLD
        
        x = self.model.ln_final(x)  # [batch_size, n_ctx, transformer.width]
        
        return x

    @torch.inference_mode()
    def text_transformer_forward(self, x: torch.Tensor, attn_mask=None):
        for i, r in enumerate(self.model.transformer.resblocks):
            if i == len(self.model.transformer.resblocks) - self.layer_idx:
                break
            #if self.model.transformer.grad_checkpointing and not torch.jit.is_scripting():
                #x = checkpoint(r, x, attn_mask)
            #else:
            
            x = r(x, attn_mask=attn_mask)
            
        return x

    #--------------------------------------------------------------
    
    def get_config(self, save_path=None, without_metadata=False):
        return super().get_config(save_path=None, without_metadata=without_metadata)
    
    def store_model(self, config_path: str, save_path: str=None, without_metadata=False):        
        super().store_model(config_path, save_path=None, without_metadata=without_metadata)

    @staticmethod
    def from_config(config, device: torch.device, save_path: str=None):  
        config["save_path"] = None
        return ConfigModel.from_config(config, device, save_path=None)        

In [ ]:
device = infer_torch_device()
a = FrozenOpenCLIPEmbedder().to(device)

[INFO]: Cuda device has a capability of 8.6 (>= 8), allowing tf32 matmul.


In [ ]:
p="[1, 2, 2]", "[1, 2, a 2]"
a.tokenize_and_push_to_device(p)

tensor([[49406,   314,   272,   267,   273,   267,   273,   316, 49407,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [49406,   314,   272,   267,   273,   267,   320,   273,   316, 49407,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0, 

In [ ]:
a.tokenize_and_push_to_device("").shape

torch.Size([1, 77])

In [ ]:
a.tokenize_and_push_to_device(["1,1,2", "2,2,2"]).shape

torch.Size([2, 77])

In [ ]:
a.model.attn_mask.shape

torch.Size([77, 77])

In [ ]:
c = a.tokenize_and_push_to_device(["1,1,2", "2,2,2"])
enc = a(c)
enc.shape, enc

(torch.Size([2, 77, 512]),
 tensor([[[-0.3819, -0.3694, -0.0712,  ...,  0.0958, -0.0834, -0.0929],
          [-0.2665,  0.1840, -0.5888,  ...,  0.7207, -1.7479,  1.2859],
          [-0.9813, -0.6659,  0.2100,  ..., -0.4228,  0.5374,  0.8488],
          ...,
          [-0.0302,  1.3877,  0.3986,  ...,  0.2663, -0.1264, -1.3759],
          [-0.0793,  1.4047,  0.3585,  ...,  0.2325, -0.0762, -1.3315],
          [ 0.1596,  1.5992,  0.2774,  ...,  0.1208, -0.1303, -1.5472]],
 
         [[-0.3819, -0.3694, -0.0712,  ...,  0.0958, -0.0834, -0.0929],
          [-1.2511,  1.4713,  0.7262,  ...,  1.1487, -0.4976,  0.4495],
          [-1.2653, -0.3404,  0.9427,  ...,  0.1537,  0.0260,  0.4574],
          ...,
          [-0.0698,  1.4014,  0.4691,  ...,  0.2275, -0.0690, -1.3637],
          [-0.1190,  1.4172,  0.4266,  ...,  0.1950, -0.0225, -1.3243],
          [ 0.1392,  1.6179,  0.3527,  ...,  0.0764, -0.0845, -1.5251]]], device='cuda:0'))

In [ ]:
a.tokenizer.decode(c[1].tolist())

'<start_of_text>2 , 2 , 2 <end_of_text>!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

In [ ]:
open_clip.decode(c[1])

'<start_of_text>2 , 2 , 2 <end_of_text>!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

## Cached model

Model takes now also (batched) scalar int values that are defined to unique conditions like $[1,2,2]=4$. If input is now such int the output is the cached pre-embedded tensor. If a non int, like a token string is passed we just do the normal embedding live.

In [ ]:
#| export
@dataclass
class CachedFrozenOpenCLIPEmbedderConfig(FrozenOpenCLIPEmbedderConfig):
    enable_cache_token_limit: bool

In [ ]:
#| export
class CachedFrozenOpenCLIPEmbedder(FrozenOpenCLIPEmbedder):
    """Adds caching support to `FrozenOpenCLIPEmbedder`."""

    def __init__(self, arch="ViT-B-32", version="datacomp_xl_s13b_b90k", max_length=77, freeze=True, layer="penultimate", enable_cache_token_limit: bool = True, **kwargs):
        super().__init__(arch=arch, version=version, max_length=max_length, freeze=freeze, layer=layer, **kwargs)  
        self.enable_cache_token_limit = enable_cache_token_limit

        self.params_config = CachedFrozenOpenCLIPEmbedderConfig(arch, version, max_length, freeze, layer, enable_cache_token_limit)
    
    def get_token_count(self, tokens, padding_token=0):
        # tokens .. [b, seq]
        collabsed_tokens = (tokens != padding_token).to(torch.int32)
        return torch.count_nonzero(collabsed_tokens, dim=-1)  # [b]
    
    @torch.inference_mode()
    def generate_cache(self, str_list: list=None, tokens=None, cached_empty_token_index=None, b_size=2048, y_on_cpu=False):       
        self.cached_empty_token_index = cached_empty_token_index       
        if exists(str_list): self.cached_tokens = self.tokenize_and_push_to_device(str_list)      
        elif exists(tokens): self.cached_tokens = tokens
        else: raise RuntimeError("please provide str_list or tokens")
        
        # note: we need to split the tokens in batches for forward pass, n gets large
        # cached_tokens     [n, 77]      ... int
        # cached_embeddings [n, 77, 512] ... float

        if self.enable_cache_token_limit:
            self.max_length = self.get_token_count(self.cached_tokens).max().item()
            self.params_config.max_length               = self.max_length
            self.params_config.enable_cache_token_limit = self.enable_cache_token_limit
            print(f"[INFO]: - `generate_cache` infered a TOKEN limit of {self.max_length}")

            #self.cached_tokens = self.cached_tokens[:, :self.max_length]
        
        n = self.cached_tokens.shape[0]
        
        n_chunks = int(np.ceil(n / b_size))
        
        in_device = self.cached_tokens.device
                
        last_ind = 0
        for i, cached_tokens in tqdm(enumerate(self.cached_tokens.chunk(n_chunks)), total=n_chunks):
            
            x = super().forward(cached_tokens.to(self.device))  # ... [b, seq, ch]
            
            if i == 0:
                mem = n * x.shape[1] * x.shape[2] * x.element_size() * 1e-9
                print(f"[INFO]: caching trying to allocate memory {(n, x.shape[1], x.shape[2])} on {'cpu' if y_on_cpu else self.device}, approx. {mem:.3f} GB")
                self.cached_embeddings = torch.zeros((n, x.shape[1], x.shape[2]), device="cpu" if y_on_cpu else self.device, dtype=x.dtype) # alloc huge memory !!
                
            self.cached_embeddings[last_ind:last_ind+x.shape[0]] = x.to(self.cached_embeddings.device)
            
            last_ind += x.shape[0]

        if self.enable_cache_token_limit:
            self.cached_embeddings = self.cached_embeddings[:, :self.max_length]
        
        if not y_on_cpu:
            self.cached_embeddings = self.cached_embeddings.to(in_device) 

    @torch.inference_mode()
    def look_up_cos_sim_cached_index(self, str_list: list=None, tokens=None):
        if exists(str_list): tokens = self.tokenize_and_push_to_device(str_list)      
        else: raise RuntimeError("please provide str_list or tokens")
                                         
        emb   = super().forward(tokens.to(self.device))
        c_emb = self.cached_embeddings
        #-----------------
        # do cos sim search
        
        emb   = emb.flatten(start_dim=1)   # [m, seq*ch]
        c_emb = c_emb.flatten(start_dim=1) # [n, seq*ch]

        norm_emb   =   emb / torch.linalg.vector_norm(  emb, dim=1, keepdim=True)
        norm_c_emb = c_emb / torch.linalg.vector_norm(c_emb, dim=1, keepdim=True) 
 
        sim     = torch.matmul(norm_c_emb, norm_emb.T) # matmul out is [n, m]
        max_idx = torch.argmax(sim, dim=0)             # reduce the c_emb dim, [m]
     
        return max_idx       
                            
    # @torch.inference_mode()
    def forward(self, c, **kwargs):  
        in_device = c.device
        
        if   c.dim() == 1: c_emb = self.cached_embeddings[c.to(self.cached_embeddings.device)].to(in_device)         #list of ints       
        elif c.dim() == 2: c_emb = super().forward(c.to(self.device))   #tokenized input      
        else: raise NotImplementedError("")

        if self.enable_cache_token_limit:
            c_emb = c_emb[:, :self.max_length]
        
        return c_emb

In [ ]:
a = CachedFrozenOpenCLIPEmbedder(enable_cache_token_limit=True).to(device)
p = ["1,1,2", "2,2,2", "4,4,4", "6,4,7", "6,4,8", "6,4,9", "6,4,1"]

a.generate_cache(p)

[INFO]: - `generate_cache` infered a TOKEN limit of 7


  0%|          | 0/1 [00:00<?, ?it/s]

[INFO]: caching trying to allocate memory (7, 77, 512) on cuda, approx. 0.001 GB


In [ ]:
a.params_config

CachedFrozenOpenCLIPEmbedderConfig(arch='ViT-B-32', version='datacomp_xl_s13b_b90k', max_length=7, freeze=True, layer='penultimate', enable_cache_token_limit=True)

In [ ]:
c_cached   = torch.tensor([0, 0, 1], device=a.device)#.cpu()
c_uncached = a.tokenize_and_push_to_device(["1,1,2", "1,1,2", "2,2,2"])

enc_cached   = a(c_cached)
enc_uncached = a(c_uncached)#.cpu()

enc_cached.shape, enc_uncached.shape, torch.allclose(enc_cached, enc_uncached, atol=1e-3)

(torch.Size([3, 7, 512]), torch.Size([3, 7, 512]), False)

In [ ]:
enc_cached.dtype, enc_uncached.dtype

(torch.float32, torch.float32)

In [ ]:
(enc_cached[0, :4, :10]-enc_uncached[1, :4, :10]).abs().max()

tensor(0.0015, device='cuda:0')

In [ ]:
(enc_cached[0, :4, :10]-enc_uncached[2, :4, :10]).abs().max()

tensor(1.9731, device='cuda:0')

# Export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()